In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("/home/jiaruil5/personality/llm_personality/psychgenerator_opensource/data/big5_training_data.csv", header = 0, index_col = 0)
df

,message,ope_z,con_z,ext_z,agr_z,neu_z
message_id,,,,,,
tweet_143508,""";;tengo mi penchi fierron up in the air & its...",-0.300409,0.459249,0.294330,0.452765,-0.894041
tweet_425070,perhaps he loved everyone but me. nbut that ma...,0.896169,0.493197,-0.414594,0.337368,0.393312
tweet_502775,"if you have 3 coffee cups in your car, you hav...",0.213276,0.176036,0.328285,0.274713,-0.211072
tweet_600640,i'm too sleepy to start work in an hour,-0.711772,1.284832,-1.013234,0.611918,0.545335
blog_79108,i worked for a recruiter a little bit after i ...,1.837614,0.042504,-0.460709,0.061335,1.808086
...,...,...,...,...,...,...
blog_46845,hey what do u think of 'unwell' by matchbox 20...,0.076771,-0.211666,0.577826,0.861935,-0.708217
tweet_611609,my life is dedicated to school right now ready...,-1.700926,1.092124,1.216876,1.034062,-0.978274
blog_107867,bush and cheney continue to assert that there ...,2.278704,0.275908,0.107220,-1.885664,-0.970312


In [2]:
df = df.sample(frac=1, random_state=42)

In [3]:
df[['ope_z', 'con_z', 'ext_z', 'agr_z', 'neu_z']].describe()

,ope_z,con_z,ext_z,agr_z,neu_z
count,8.463040e+05,8.463040e+05,8.463040e+05,8.463040e+05,8.463040e+05
mean,-4.728533e-17,-2.757192e-17,-1.863875e-17,1.193048e-17,1.726183e-17
std,9.999994e-01,9.999994e-01,9.999994e-01,9.999994e-01,9.999994e-01
min,-1.791328e+01,-1.606235e+01,-1.647491e+01,-2.421485e+01,-9.958985e+00
25%,-5.990037e-01,-5.480083e-01,-5.593109e-01,-4.808781e-01,-5.858125e-01
50%,-6.985836e-02,5.923059e-02,4.827777e-03,1.119149e-01,-1.141641e-01
75%,5.346785e-01,5.938862e-01,5.516231e-01,5.968365e-01,4.446708e-01
max,1.507327e+01,2.519352e+01,2.000737e+01,1.254866e+01,3.199032e+01


In [4]:
def label_by_percentile(value, p33, p67):
    if value <= p33:
        return 0
    elif value > p67:
        return 1
    else:
        return -1

def dataset_filter(df):
    z_columns = [col for col in df.columns if col.endswith('_z')]
    
    for col in z_columns:
        p33 = df[col].quantile(1/3)
        p67 = df[col].quantile(2/3)
        print(f"Column: {col}, the lower threshold: {p33}, the higher threshold: {p67}")
        df[f'{col}_label'] = df[col].apply(lambda x: label_by_percentile(x, p33, p67))
    return df

filtered_df = dataset_filter(df)

Column: ope_z, the lower threshold: -0.4029575700493231, the higher threshold: 0.3034446294558673
Column: con_z, the lower threshold: -0.31693058187339396, the higher threshold: 0.3969570757335582
Column: ext_z, the lower threshold: -0.34795953512667344, the higher threshold: 0.3459400121747427
Column: agr_z, the lower threshold: -0.24894595714293216, the higher threshold: 0.42143799743769655
Column: neu_z, the lower threshold: -0.4153174549213307, the higher threshold: 0.2193972458545086


In [5]:
print(filtered_df.isnull().sum())

message        0
ope_z          0
con_z          0
ext_z          0
agr_z          0
neu_z          0
ope_z_label    0
con_z_label    0
ext_z_label    0
agr_z_label    0
neu_z_label    0
dtype: int64


In [6]:
filter_df = filtered_df.drop(columns=["ope_z", "con_z", "ext_z", "agr_z", "neu_z"])
filter_df

,message,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
message_id,,,,,,
tweet_79485,afternoon beer and football. love bank hol wee...,0,1,1,1,0
blog_285431,i'm not much of a curser.i have a pretty exten...,1,0,0,-1,1
tweet_592581,spent the entire day with my momma. and it was...,1,1,1,1,-1
blog_243831,"i layed in bed last night, restlessly tossing ...",1,0,0,-1,1
blog_232829,in the a.m.this has to be the sixteenth effort...,1,-1,-1,1,1
...,...,...,...,...,...,...
tweet_156286,playing arkham asylum. i am the bat.,-1,-1,0,-1,0
tweet_176785,the last 15 seconds i mean 10 minutes of this ...,0,1,-1,-1,1
blog_372010,"as you may have noticed, my post titles have b...",-1,0,0,0,0


In [7]:
from sklearn.model_selection import train_test_split
for col in ['ope_z_label', 'con_z_label', 'ext_z_label', 'agr_z_label', 'neu_z_label']:
    df = filtered_df[filtered_df[col] != -1]
    train_df, test_df = train_test_split(df, test_size=0.05, random_state=42)
    train_df.to_csv(f"/data/user_data/wenkail/llm_personality/data/big5_data_generator_{col[0]}_train.csv", index=True)
    test_df.to_csv(f"/data/user_data/wenkail/llm_personality/data/big5_data_generator_{col[0]}_test.csv", index=True)

In [10]:
df = pd.read_csv("/data/user_data/wenkail/llm_personality/data/big5_data_generator_n_test.csv")
df

,message_id,message,ope_z,con_z,ext_z,agr_z,neu_z,ope_z_label,con_z_label,ext_z_label,agr_z_label,neu_z_label
0,blog_260050,okay everyone. sitting here in french class. a...,-0.726706,1.429533,0.815121,0.736769,-1.869206,0,1,1,1,0
1,blog_540050,goal #1- lose 10% (27.3 pounds) done june 200...,-0.282830,0.290903,0.850489,-0.256927,-1.060859,-1,-1,1,0,0
2,blog_231330,the radical free-market types tend not to get ...,1.037915,0.587940,0.291303,-0.239140,-0.656103,1,1,-1,-1,0
3,tweet_531601,these niggas act like hoes behind close doors,-0.028448,1.835111,-0.126631,-0.453835,-1.085052,-1,1,-1,0,0
4,blog_674421,hiatus i've been rather moody lately.i'd exp...,1.536522,-1.778976,-1.368184,-1.717173,1.344149,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
28206,blog_564190,"ahhh, so after a talk with a friend and a week...",0.325578,0.824152,2.455467,1.462900,-0.777602,1,1,1,1,0
28207,blog_219962,best mc ever: mf doom ......plus this muthaf...,0.514370,-0.217161,-0.037770,-0.292992,-0.511092,1,-1,-1,0,0
28208,blog_44255,i am also expecting that god will be my guide....,0.356987,0.645025,-1.131896,0.558180,-0.449473,1,1,0,1,0
28209,tweet_76570,dakota sleeps 6-8 hours a night already. it's ...,-0.741915,0.998589,0.856650,1.562575,-0.416486,0,1,1,1,0
